In [ ]:
import numpy as np
import pandas as pd
import datetime
from numpy.linalg import inv
from pandas_datareader import data as pdr
import yfinance as yfin
#import matplotlib.pyplot as plt
#from scipy.optimize import minimize
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
start = datetime.datetime(2010,1,1)
end = datetime.datetime.now()
yfin.pdr_override()


In [ ]:
def data_returns(data): #generating daily avg assets return 
    returns = (data/data.shift(1))-1
    return returns

In [ ]:
def portfolio_covariance(data): #generating yearly avg portfolio risk
    yearly_port_cov = data_returns(data).cov()*252
    return yearly_port_cov
    

#### Markowitz Portfolio Theory Function

In [ ]:
# 1000 combination with function (Simple return)

def combination(Ticker):

    #load data and create random weights dataframe
    data = pd.DataFrame()
    data = pdr.get_data_yahoo(Ticker,start,end)['Adj Close']
    
    weights_array = pd.DataFrame(columns=[Ticker])

    #empty array for Risk and Return Columns
    port_return = []
    port_risk = []
  
    #generating 1000 random combination for the portfolio
    for w in range (1000):
        #get random weights for each Ticker
        weights = np.random.random(len(Ticker)) 

        #weights return array
        weights = weights/np.sum(weights) 
        
        #insert weights (array) to a dataframe called each_weights
        weights_array.loc[len(weights_array)] = weights.tolist()  
        
        #calculate avg annual return and store it to array port_return
        port_return.append(np.sum(weights * (data_returns(data).mean()*252)))
        
        #calculate avg annual std dev/risk and store it to array port_risk
        port_risk.append(np.sqrt(np.dot(weights.T, np.dot(portfolio_covariance(data), weights)))) 


    #create dataframe called portfolios to store risk and return
    portfolios = pd.DataFrame({'Return':port_return, 'Risk': port_risk}) 
    for i in range(len(Ticker)):   
        weights_array.columns.values[i] = Ticker[i]
        
    result = pd.concat([weights_array, portfolios], axis=1) #concat dataframe each_weights and portfolios
    
    #return port_return, port_risk
    return  result

In [ ]:
#Calculation Minimum Variance Portfolio with Lagrange multiplier and Cramer's Rule based on Modul Portfolio Selection from Prof. Dr. Andreas Görg
def Min_Varianz(Ticker):
    
    data = pd.DataFrame()
    data = pdr.get_data_yahoo(Ticker,start,end)['Adj Close']

    #Variable
    Mu = (data_returns(data).mean()*252).to_numpy()
    Sigma = (portfolio_covariance(data)).to_numpy()
    One = [1 for x in range(len(Mu))]
    One_arr = np.array(One)
    Sigma_inverse = inv(Sigma)

    a = np.dot(np.dot(Mu.T,Sigma_inverse),Mu)
    b = np.dot(np.dot(Mu.T,Sigma_inverse),One_arr)
    c = np.dot(np.dot(One_arr,Sigma_inverse),One_arr)


    Return_x = b/c
    Risk_x = 1/np.sqrt(c)

    weights = (np.dot((Return_x*c-b)*Sigma_inverse,Mu)+(np.dot((a-Return_x*b)*Sigma_inverse,One_arr)))/(a*c-b**2)
    
    Min_Port = np.array([Return_x,Risk_x])

    Data = np.concatenate((weights,Min_Port))

    Columns = Ticker
    Columns.append('Return')
    Columns.append('Risk')

    Min_Varianz_Data = pd.DataFrame(columns=Columns)
    Min_Varianz_Data.loc[len(Min_Varianz_Data)] = Data    


    return Min_Varianz_Data

#### Calling Function

In [ ]:
xxx = ['AAPL','TSLA','COKE']

In [7]:
my_combination = combination(xxx)

In [8]:
my_combination['Sharpe Ratio'] = (my_combination['Return']-0.0304)/my_combination['Risk']
my_combination

,AAPL,TSLA,COKE,Return,Risk,Sharpe Ratio
0,0.480094,0.507443,0.012463,0.265547,0.248315,0.946971
1,0.092799,0.435727,0.471473,0.389606,0.341943,1.050486
2,0.225112,0.557789,0.217099,0.317730,0.274692,1.046008
3,0.304659,0.523833,0.171508,0.306968,0.262397,1.054002
4,0.060165,0.242718,0.697118,0.456581,0.429647,0.991933
...,...,...,...,...,...,...
995,0.153162,0.336831,0.510007,0.403547,0.352408,1.058847
996,0.294744,0.379660,0.325596,0.353173,0.289593,1.114571
997,0.607299,0.252042,0.140659,0.309130,0.253401,1.099954
998,0.023907,0.448816,0.527277,0.403861,0.364670,1.024108


In [9]:
Sharpe_max = my_combination[my_combination['Sharpe Ratio']== my_combination['Sharpe Ratio'].max()]
Sharpe_max

,AAPL,TSLA,COKE,Return,Risk,Sharpe Ratio
670,0.452825,0.278896,0.268279,0.341846,0.274148,1.136049


In [10]:
Min_Var_Portfolio = Min_Varianz(xxx)
Min_Var_Portfolio

[*********************100%***********************]  3 of 3 completed


,AAPL,TSLA,COKE,Return,Risk
0,0.578681,0.383732,0.037587,0.276886,0.243822


In [11]:
Min_Var_Portfolio['Sharpe Ratio'] = (Min_Var_Portfolio['Return']-0.0304)/Min_Var_Portfolio['Risk']
Min_Var_Portfolio


,AAPL,TSLA,COKE,Return,Risk,Sharpe Ratio
0,0.578681,0.383732,0.037587,0.276886,0.243822,1.010925


In [12]:
#More Simple Version of Minimum Variance

# Find index of row with minimum risk
min_risk_idx = my_combination['Risk'].idxmin()

# Subset DataFrame to only include rows with minimum risk
min_risk_df = my_combination[my_combination['Risk'] == my_combination.loc[min_risk_idx, 'Risk']]

# Find index of row with maximum return among rows with minimum risk
optimal_return_idx = min_risk_df['Return'].idxmax()

# Subset DataFrame to only include row with both minimum risk and maximum return
optimal_portfolio = min_risk_df[min_risk_df['Return']==min_risk_df.loc[optimal_return_idx,'Return']]

In [13]:
#reformat data
def reformat_Data(Data):
     
     Data_1 = Data.copy()
     Data_1.iloc[:,:-3] = (Data_1.iloc[:,:-3]*100).round(2).astype(str) + "%"
     Data_1.iloc[:,-3:] = (Data_1.iloc[:,-3:]*100).round(2)
     
     return Data_1


In [14]:

Scatter_Data = reformat_Data(my_combination)
Min_Varianz_Data = reformat_Data(Min_Var_Portfolio)
Sharpe_max_Data = reformat_Data(Sharpe_max)
Optimal_Portfolio_Data = reformat_Data(optimal_portfolio)

In [18]:
sorted = Scatter_Data.sort_values(by=['Risk'])
test = sorted.groupby(np.arange(len(sorted))//8).max()
test

,AAPL,TSLA,COKE,Return,Risk,Sharpe Ratio
0,62.69%,41.55%,5.36%,28.14,24.46,102.86
1,64.25%,44.99%,8.89%,29.03,24.57,105.87
2,66.86%,46.4%,9.53%,29.30,24.59,106.83
3,68.7%,47.48%,9.16%,28.98,24.66,105.34
4,70.04%,49.03%,9.57%,29.50,24.75,107.17
...,...,...,...,...,...,...
120,9.51%,9.21%,72.04%,46.69,44.08,99.47
121,9.4%,6.57%,73.97%,47.18,44.92,98.91
122,6.41%,8.01%,77.05%,48.08,46.38,98.14
123,9.4%,5.37%,80.2%,48.84,47.73,96.65


In [19]:
sorted.groupby(np.arange(len(sorted))//8)['Sharpe Ratio'].agg(['max'])

,max
0,102.86
1,105.87
2,106.83
3,105.34
4,107.17
...,...
120,99.47
121,98.91
122,98.14
123,96.65


#### Visualization

In [20]:
#Visualizing the 1000 combination data
fig1= px.scatter(data_frame=Scatter_Data,
               x='Risk',y='Return',hover_data=Scatter_Data.columns, color='Sharpe Ratio',
               )

#Visualizing a portfolio with lowest risk
fig2 = px.scatter(data_frame=Min_Varianz_Data, x = 'Risk', y = 'Return',hover_data=Min_Varianz_Data.columns,labels=dict(Risk="Risk (%)", Return="Return (%)"))
#Visualizing a portfolio with highest sharpe ratio
fig3 = px.scatter(data_frame=Sharpe_max_Data, x = 'Risk', y = 'Return',hover_data=Sharpe_max_Data.columns,labels=dict(Risk="Risk (%)", Return="Return (%)"))

fig4 =px.scatter(data_frame=Optimal_Portfolio_Data,x = 'Risk', y = 'Return',hover_data=Optimal_Portfolio_Data.columns,labels=dict(Risk="Risk (%)", Return="Return (%)"))

#adding marker to fig2 and fig3
fig2.update_traces(marker=dict(size=15, color="Red",symbol="x"))
fig3.update_traces(marker=dict(size=15, color="Green",symbol="x"))
fig4.update_traces(marker=dict(size=15, color="Blue",symbol="x"))


#here i'm trying to draw a efficient frontier line from the visualization but still not getting the result i wanted
fig5= px.line(data_frame=test,
               x='Risk',y='Return'
               )


fig = go.Figure(data = fig1.data + fig2.data + fig3.data+fig4.data+fig5.data)
fig.update_layout(
    title="Portfolio Combination",
    xaxis_title="Risk (%)",
    yaxis_title="Return (%)",
    font=dict(
        family="Bahnschrift",
        size=18,
        color="RebeccaPurple"
    )
)
fig.show()